# BRENT CRUDE OIL PRICE FORECASTING (SARIMA and LSTM models)

In [9]:
# Required modules

# For common operations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For SARIMA model
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose # For time series decomposition
from pmdarima import auto_arima


# For LSTM model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import load_model  # Allows load a previously saved model.

# To evaluate the models
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# To enable interactive plots

## For Jupyter web (requires ipympl module)
#%matplotlib widget

## For IDEs, like PyCharm
import matplotlib
matplotlib.use('nbagg')


### Data loading

In [14]:
full_data = pd.read_csv('../data/brent_daily_prices.csv', parse_dates=['DATE'], date_format='%d/%m/%Y', index_col='DATE', na_values='.')  # In the original time series, NA values are represented by a period (.)
data = full_data['DCOILBRENTEU']
data.rename_axis('date', inplace=True)
data.rename('brent_crude_oil', inplace=True)
data.fillna(method='ffill', inplace=True)  # Replace NaN values with the last valid observation.
data.index = pd.DatetimeIndex(data.index).to_period('B').to_timestamp()  # Sets the frequency for the time series.
data

C:\Users\hp\AppData\Local\Temp\ipykernel_8684\3791287256.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)  # Replace NaN values with the last valid observation.
C:\Users\hp\AppData\Local\Temp\ipykernel_8684\3791287256.py:6: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  data.index = pd.DatetimeIndex(data.index).to_period('B').to_timestamp()  # Sets the frequency for the time series.


date
1987-05-20    18.63
1987-05-21    18.45
1987-05-22    18.55
1987-05-25    18.60
1987-05-26    18.63
              ...  
2023-08-01    85.34
2023-08-02    84.01
2023-08-03    86.19
2023-08-04    87.38
2023-08-07    86.47
Freq: B, Name: brent_crude_oil, Length: 9449, dtype: float64

### Exploratory Data Analysis

In [15]:
# Basic EDA of the data
print(data.describe())
print('\n')
print(data.info())
print('Missing values: ', data.isna().sum())

# Plots the data
plt.figure(figsize=(10, 4))
plt.plot(data)
plt.title('Brent Crude Oil Price since 1987')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.show()

count    9449.000000
mean       49.155899
std        32.956536
min         9.100000
25%        19.150000
50%        40.800000
75%        72.050000
max       143.950000
Name: brent_crude_oil, dtype: float64


<class 'pandas.core.series.Series'>
DatetimeIndex: 9449 entries, 1987-05-20 to 2023-08-07
Freq: B
Series name: brent_crude_oil
Non-Null Count  Dtype  
--------------  -----  
9449 non-null   float64
dtypes: float64(1)
memory usage: 147.6 KB
None
Missing values:  0


<IPython.core.display.Javascript object>

In [16]:
# Checking the composition of the data
seasonal_decompose(data).plot()
plt.xticks(rotation=45)
plt.show()

<IPython.core.display.Javascript object>

### Data splitting

In [17]:
train_data = data.iloc[:len(data) - 30]
test_data = data.iloc[len(data) - 30:]

## SARIMA model

### Getting the parameters for the model

In [18]:
# Runs auto_arima function to get the parameters for the SARIMA model
opt_model = auto_arima(train_data, maxiter=100, trace=True)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=30304.148, Time=10.92 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=30327.930, Time=0.39 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=30317.381, Time=0.77 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=30316.773, Time=0.88 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=30326.151, Time=0.34 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=30314.723, Time=6.53 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=30314.425, Time=4.24 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=30303.250, Time=7.19 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=30310.245, Time=4.26 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=30320.821, Time=15.32 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=30305.257, Time=11.04 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=30303.251, Time=10.98 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=30308.252, Time=4.49 sec
 ARIMA(4,1,3)(0,0,0)[0] intercept   : AIC=30307.202, Time=6.75 sec
 ARIMA(3,1,2)(0

### Model training

In [19]:
# Trains the model according to the auto_arima output -> (2,1,3)x(0,0,0,0)
sarima_model_eval = SARIMAX(train_data, order=(2, 1, 3), seasonal_order=(0, 0, 0, 0))
estimator_eval = sarima_model_eval.fit()

# Gets forecast for evaluation
preds = estimator_eval.forecast(len(test_data))

# Plot the results
test_data.plot(color='blue', label='Actual')
preds.plot(color='green', label='Forecasts (30 days)')

plt.title('Brent Crude Oil Price Forecast (SARIMA model evaluation)')
plt.xlabel('Date')
plt.ylabel('Oil price (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### Model evaluation

In [20]:
rmse = np.sqrt(mean_squared_error(test_data.values, preds.values))
mae = mean_absolute_error(test_data.values, preds.values)
mape = mean_absolute_percentage_error(test_data.values, preds.values)

print('Root Mean Square Error (RMSE): {} \nMean Absolute Error (MAE): {} \nMean Absolute Percentage Error (MAPE): {}'. format(np.round(rmse, 3), np.round(mae, 3), np.round(mape, 3)))

Root Mean Square Error (RMSE): 7.543 
Mean Absolute Error (MAE): 6.431 
Mean Absolute Percentage Error (MAPE): 0.078


### Using the model to forecasts Brent crude oil price for the following 15 days

In [21]:
# Sets the model
sarima_model_forecast = SARIMAX(data, order=(2,1,3), seasonal_order=(0,0,0,0))
estimator_forecast = sarima_model_forecast.fit()

# Makes predictions
steps_ahead = 15
forecasts = estimator_forecast.forecast(steps_ahead)
ci = estimator_forecast.conf_int()

# Displays the results
short_data = data[data.index.year >= 2023]
short_data.plot(color='blue', label='Actual')
forecasts.plot(color='red', label='Forecasts')

plt.title('Brent Crude Oil Price Forecast with SARIMA (15 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

print('Forecasts for the following {} days: \n'.format(steps_ahead))
print(forecasts)

<IPython.core.display.Javascript object>

Forecasts for the following 15 days: 

2023-08-08    86.307389
2023-08-09    86.385525
2023-08-10    86.422531
2023-08-11    86.357448
2023-08-14    86.391188
2023-08-15    86.394938
2023-08-16    86.376393
2023-08-17    86.389151
2023-08-18    86.387762
2023-08-21    86.382943
2023-08-22    86.387353
2023-08-23    86.386137
2023-08-24    86.385046
2023-08-25    86.386461
2023-08-28    86.385860
Freq: B, Name: predicted_mean, dtype: float64


## LSTM model

### Data preparation

In [22]:
# Reshapes the data to feed the model
full_data_lstm = data.values.reshape(-1, 1)
train_data_lstm = train_data.values.reshape(-1, 1)
test_data_lstm = test_data.values.reshape(-1, 1)

# Defines train and test sets
X_train = []
y_train = []
ws = 30 # Window size: indicates the number of previous time steps. The more, may lead to higher accuracy, but increases complexity and training time.

for i in range(ws, len(train_data_lstm)):
    X_train.append(train_data_lstm[i - ws: i])
    y_train.append(train_data_lstm[i])

X_train, y_train = np.array(X_train), np.array(y_train)

### Model training
The model hyperparameters were chosen after evaluating many different combinations.

In [23]:
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape = (X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')  
model.fit(X_train, y_train, epochs=100, batch_size=600)

c:\Users\hp\KAIM\KAIM-W10\.week10\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 195ms/step - loss: 8906.3828
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - loss: 290606.4375
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - loss: 3227427.2500
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - loss: 24450568.0000
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - loss: 1138792.8750
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 140ms/step - loss: 373213.1562
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - loss: 12454.3652
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - loss: 2112.4336
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 154ms/step - loss: 856.1956
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - loss: 584.5531
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 150ms/step - loss: 548.2406
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - loss: 525.4077
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 154ms/step - loss: 524.2949
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - loss: 5

### Plotting loss
This is useful to check if the number of epochs is adequate: 
- A flat trend at the end of the curve is desired.
- If the end of the curve has a downward trend, it could indicate an opportunity of improvement, requiring a larger number of epochs.
- If the end of the curve has an upward trend, it could indicate overfitting, so fewer epochs are required.

In [24]:
plt.plot(range(len(model.history.history['loss'])), model.history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

<IPython.core.display.Javascript object>

### Saving the trained model

In [25]:
model.save('model1')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=model1.

### Loading a model
Please, load the model "brent_price_forecast_lstm_model" to evaluate it or make forecasts.

That model was trained with the following parameters:
- epochs = 100
- units = 150 (indicates the number of neurons in the LSTM layer)
- batch_size = 600
- activation = 'relu' (indicates the activation function in the LSTM layer)
- optimizer = 'adam' 
- loss = 'mse'
- The data to feed the model was prepared with a window size of 30.

Neural network algorithms are stochastic, which means they make use of randomness, such as initializing to random weights, and in turn the same network, with the same hyperparameters, trained on the same data can produce different results [1]. This is the reason why it is necessary to load the aforementioned model.



[1] https://machinelearningmastery.com/stochastic-in-machine-learning

In [26]:
model = load_model('brent_price_forecast_lstm_model')

ValueError: File format not supported: filepath=brent_price_forecast_lstm_model. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(brent_price_forecast_lstm_model, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

### Model testing

In [20]:
prediction_set = []
batch_one = train_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))

for i in range(len(test_data)):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set] # Transforms a list of arrays into a list of single float items.
predictions = pd.Series(prediction_set, index=test_data.index)


# Plots the results
test_data.plot(color='blue', label='Actual')
predictions.plot(color='green', label='Prediction')

plt.title('Brent Crude Oil Price Forecast (LSTM model evaluation)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### Model evaluation

In [21]:
rmse = np.sqrt(mean_squared_error(test_data_lstm, predictions))
mae = mean_absolute_error(test_data_lstm, predictions)
mape = mean_absolute_percentage_error(test_data_lstm, predictions)
print('Root Mean Square Error (RMSE): {} \nMean Absolute Error (MAE): {} \nMean Absolute Percentage Error (MAPE): {}'. format(np.round(rmse, 3), np.round(mae, 3), np.round(mape, 3)))

Root Mean Square Error (RMSE): 3.677 
Mean Absolute Error (MAE): 3.224 
Mean Absolute Percentage Error (MAPE): 0.041


### Using the model to forecast the Brent crude oil price for the following 30 days

In [22]:
# Makes the predictions 
prediction_set = []
batch_one = full_data_lstm[-ws:]
new_batch = batch_one.reshape((1, ws, 1))
days_to_forecast = 30

for i in range(days_to_forecast):
    pred = model.predict(new_batch, verbose=False)[0]
    prediction_set.append(pred)
    new_batch = np.append(new_batch[:, 1:, :], [[pred]], axis=1)

prediction_set = [i[0] for i in prediction_set]  # Transforms a list of arrays into a list of single float items.
date_range = pd.date_range(test_data.index[-1], periods=days_to_forecast, freq='B')   
forecast = pd.Series(prediction_set, index=date_range, name='Forecast')


# Displays results
short_data = data.iloc[-250:] # Last n datapoints of the original time series.

short_data.plot(color='blue', label='Actual')
forecast.plot(color='red')

plt.title('Brent Crude Oil Price Forecast with LSTM (30 days ahead)')
plt.xlabel('Date')
plt.ylabel('Price per barrel (USD)')
plt.legend()
plt.show()

print('Forecasts for the following {} days: '.format(days_to_forecast))
print(forecast)

<IPython.core.display.Javascript object>

Forecasts for the following 30 days: 
2023-08-07     86.838974
2023-08-08     88.119522
2023-08-09     88.610596
2023-08-10     88.780853
2023-08-11     90.181458
2023-08-14     90.257935
2023-08-15     90.509865
2023-08-16     90.808083
2023-08-17     90.853645
2023-08-18     89.893936
2023-08-21     90.618149
2023-08-22     92.039726
2023-08-23     93.088234
2023-08-24     92.772865
2023-08-25     93.825233
2023-08-28     95.939301
2023-08-29     95.710098
2023-08-30     94.572220
2023-08-31     95.415863
2023-09-01     95.783859
2023-09-04     96.019470
2023-09-05     95.710281
2023-09-06     95.871262
2023-09-07     96.260857
2023-09-08     96.980095
2023-09-11     97.948135
2023-09-12     98.919952
2023-09-13     98.961479
2023-09-14    100.141144
2023-09-15    101.966057
Freq: B, Name: Forecast, dtype: float32


This LSTM model was trained to forecast the Brent crude oil price for 30 days ahead from the last date in the time series. That means the model could produce extremely erroneous results if it is used with long time spans (variable _days_to_forecast_), for example, 60 days or more.

## Conclusions
1. Modeling crude oil prices is a complicated task due to the high variation and volatility associated with its market. However, it's necessary to do so, as oil is one of the most important energies driving the world economy and represents a crucial factor in most industries.
   
2. In the evaluation of the SARIMA model, a Root Mean Squared Error (RMSE) of 7.543 and a Mean Absolute Error (MAE) of 6.431 were obtained. The forecast graph for the next 15 days shows a horizontal line with a value of approximately 83.5. This means that the SARIMA model estimates the Brent crude oil price for the next 15 days to be around $83.5 per barrel, on average.

3. In the evaluation of the LSTM model, a Root Mean Squared Error (RMSE) of 3.677 and a Mean Absolute Error (MAE) of 3.224 were obtained. The forecast graph for the next 30 days demonstrates its capability to capture the trend and shape of the time series of actual data.

4. Considering the errors obtained in the evaluations of each model, along with their ability to capture the trend and shape of the original time series, it's evident that the LSTM model performs significantly better in forecasting the Brent crude oil price.